# Probability distribution derivator_v2

#### 파라미터 변경
#### 20221019. 분포형태 추가(파라미터 값 계산 가능한 것으로)

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats

In [2]:
delay_pd = pd.read_csv('dataset_v4/리스크 기초자료_1차(1~7)_20221013_csv.csv')
delay = []
pre_data = []
pre_temp = []
cal_data = []
cal_temp = []

In [3]:
l_L3 = ["토공"]
l_L4 = ["본선","IC/JC","지선/부체도로"]
l_L7_1 = ["흙깎기","흙쌓기","토공기타"]
l_L7_2 = [["토사","리핑","발파","흙깎기기타"],["노체","노상","흙쌓기기타"],["비탈면보호공","연약지반처리","보강토옹벽","콘크리트옹벽","옹벽기타","토공기타"],]
l_Risk = ["소음/진동","용지","진입로","기타","물가변동","지급신청","수급변경","업체변경","문제검토","문제협의","설계변경","시공계획","업무","검토","문화재","일반시설","유해/분묘","건설폐기물","임목폐기물","폐수","우천","태풍","폭염","폭설","인적","물적"]

In [4]:
for i in l_L4:
    for j in l_L7_1:
        for k in l_L7_2[l_L7_1.index(j)]:
            for l in l_Risk:
                search_L4 = delay_pd['L4'] == i
                s_L4 = delay_pd[search_L4]
                search_L7_1 = s_L4['L7-1'] == j
                s_L7_1 = s_L4[search_L7_1]
                search_L7_2 = s_L7_1['L7-2'] == k
                s_L7_2 = s_L7_1[search_L7_2]
                search_Risk = s_L7_2['중'] == l
                s_Risk = s_L7_2[search_Risk]
                
                pre_temp = []
                if s_Risk['중'].empty==False:
                    pre_temp.append(s_Risk['No'])
                    pre_temp.append(s_Risk['L3'])       
                    pre_temp.append(s_Risk['L4'])
                    pre_temp.append(s_Risk['L7-1'])
                    pre_temp.append(s_Risk['L7-2'])
                    pre_temp.append(s_Risk['대'])
                    pre_temp.append(s_Risk['중'])
                    pre_temp.append(s_Risk['실제공기'])
                    pre_temp.append(s_Risk['공기지연'])
                    pre_temp.append(s_Risk['공기지연'].count())
                    pre_temp.append(s_Risk['공기지연'].mean())
                    pre_temp.append(s_Risk['공기지연'].var())
                    pre_temp.append(s_Risk['공기지연'].std())
                    pre_temp.append(s_Risk['공기지연'].max())
                    pre_temp.append(s_Risk['공기지연'].min())
                    pre_data.append(pre_temp)

                cal_temp = []
                if s_Risk['중'].empty==False:
                    cal_temp.append(s_Risk['No'].values)
                    cal_temp.append(s_Risk['L3'].values)       
                    cal_temp.append(s_Risk['L4'].values)
                    cal_temp.append(s_Risk['L7-1'].values)
                    cal_temp.append(s_Risk['L7-2'].values)
                    cal_temp.append(s_Risk['대'].values)
                    cal_temp.append(s_Risk['중'].values)
                    cal_temp.append(s_Risk['실제공기'].values)
                    cal_temp.append(s_Risk['공기지연'].values)
                    cal_temp.append(s_Risk['공기지연'].count())
                    cal_temp.append(s_Risk['공기지연'].mean())
                    cal_temp.append(s_Risk['공기지연'].var())
                    cal_temp.append(s_Risk['공기지연'].std())
                    cal_temp.append(s_Risk['공기지연'].max())
                    cal_temp.append(s_Risk['공기지연'].min())
                    cal_data.append(cal_temp)

In [5]:
df=pd.DataFrame(pre_data)
df.to_csv("dataset_v3/preset_data_v3.csv", encoding='utf-8-sig')
df=pd.DataFrame(cal_data)
df.to_csv("dataset_v3/cal_data_v3.csv", encoding='utf-8-sig')

In [6]:
new_temp = []
new_data = []
#new_data = [['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류','공기','리스크발생확률\n(%)','확률분포','P-value','리스크횟수','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
dist_names = ["norm", "gamma", "beta", "expon", "t", "chi2", "uniform"]
dist_results = []
params = {}
count = 0

In [7]:
for i in range(0, len(cal_data)):
    count = 0
    new_temp = []
    count = count + cal_data[i][7][0]
    for j in range(1, len(cal_data[i][7])):
        if cal_data[i][7][j-1]!=cal_data[i][7][j]:
            count = count + cal_data[i][7][j]
    for k in range(0, len(cal_data[i])):
        if k == 7:
            new_temp.append(count) 
        elif k == 8:
            data = cal_data[i][8]
            new_temp.append(cal_data[i][9]*100/count)
            dist_results = []
            params = {}
            for dist_name in dist_names:
                dist = getattr(scipy.stats, dist_name)
                param = dist.fit(data)
                params[dist_name] = param
                D, p = stats.kstest(data, dist_name, param)
                dist_results.append((dist_name, p))
            best_dist, best_p = (max(dist_results, key=lambda item: item[1])) 
            new_temp.append(best_dist)
            new_temp.append(best_p)
        elif k >= 9:
            new_temp.append(cal_data[i][k])
        else:
            if k != 0:
                new_temp.append(cal_data[i][k][0])
    new_data.append(new_temp)


/home/min/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/min/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:400: RuntimeWarning: invalid value encountered in double_scalars
  return m3 / np.power(m2, 1.5)
/home/min/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:411: RuntimeWarning: invalid value encountered in double_scalars
  return m4 / m2**2 - 3
/home/min/.local/lib/python3.6/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


AttributeError: 'poisson_gen' object has no attribute 'fit'

In [ ]:
df=pd.DataFrame(new_data)
df.columns=[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크발생확률\n(%)','확률분포','P-value','리스크횟수','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
df=df[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크횟수','리스크발생확률\n(%)','확률분포','P-value','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
df.to_csv("dataset_v4/리스크 확률분포_20221013.csv", encoding='utf-8-sig')